# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [24]:
import datetime
import os

import openai
from dotenv import load_dotenv, find_dotenv
import warnings
import pandas as pd

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

In [29]:
_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY_FOUR']
# account for deprecation of LLM model

# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"
# print(openai.api_key)
warnings.filterwarnings('ignore')


In [14]:
df = pd.read_csv('data/Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [16]:
llm = ChatOpenAI(temperature=0.9,
                 model=llm_model)
llm.openai_api_key = openai.api_key

In [21]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Just the name."
)

In [22]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,    # Here we define to use above prompt
)

In [23]:
# Bellow we define product with whom we're going use prompt
product = "Queen Size Sheet Set"
chain.run(product)     

'Royal Bedding.'

## SimpleSequentialChain

In [25]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)
llm.openai_api_key = openai.api_key
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [26]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [27]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [28]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royal Linens"
Royal Linens is a luxury home textile brand offering high-quality bedding, bath, and kitchen linens with elegant designs and superior comfort.

> Finished chain.


'Royal Linens is a luxury home textile brand offering high-quality bedding, bath, and kitchen linens with elegant designs and superior comfort.'